# Partie 3 : Premiers pas vers les méthodes de ML supervisé en python

L'idée de cette partie est de tester différentes méthodes d'apprentissage statistique supervisées usuelles. Pour cela nous allons utiliser les données issues de la table de description des individus interviewés lors de l'enquête INCA 3 sur la consommation et les habitudes alimentaires des français.

L'objectif consistera à prédire au mieux l'IMC d'un individu grâce aux diverses informations que nous détenons sur la personne. Contrairement à la partie 2 sur le clustering, il s'agit ici de d'apprentissage supervisé car nous avons en notre possessions des données labélisées. Parmi les méthodes d'apprentissages supervisé on distingue généralement deux grandes familles que sont la classification et la régression. Ici nous sommes confronté à un problème de régression puisque nous souhaitons prédire l'indice de masse corporelle exacte. Pour cela nous allons tester plusieurs méthodes différentes afin d'analyser lesquelles sont les plus efficaces sur les données que nous possédons.


## 1. Prise en main des données

Les données de l'enquête INCA3 sont disponibles sur *Data.gouv* à l'adresse suivante : https://www.data.gouv.fr/fr/datasets/donnees-de-consommations-et-habitudes-alimentaires-de-letude-inca-3/. Cette table contient les données des questionnaires face-à-face relatifs aux volets « Socio-économique » et « Mesures anthropométriques » et des données des questionnaires auto-administrés relatifs aux volets « Etat de santé » et « Tabagisme ». 

Elle regroupe les informations suivantes : caractéristiques socio-démographiques de l’individu (ou de son représentant dans le cas des enfants), caractéristiques
socio-démographiques de la personne de référence du foyer, niveau de vie du foyer, insécurité alimentaire, caractéristiques anthropométriques (poids, taille, indice
de masse corporelle, statut pondéral) ; statut vis-à-vis d’allergies ou d’intolérances alimentaires, types de régimes alimentaires, types d’allergies ou d’intolérances
alimentaires, régimes et histoire pondérale, statut vis-à-vis de la grossesse, de l’allaitement et de la ménopause (uniquement pour les femmes de 15 ans et plus),
statut tabagique ; indicateurs de sous ou sur-déclaration en termes de consommations alimentaires.

Nous avons préalablement selectionné un grand nombre de variables issues de cette base que nous avons ensuite enregistré dans un bucket s3. Vous pouvez les télécharger grâce à ma commande suivante :

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from pyarrow import fs

In [ ]:
s3 = fs.S3FileSystem(endpoint_override='https://'+'minio.lab.sspcloud.fr')

bucket = "projet-funathon"
path_data =  "2023/sujet3/diffusion/description_individu_inca.parquet"

df = pq.ParquetDataset(f'{bucket}/{path_data}', filesystem=s3).read_pandas().to_pandas()

In [ ]:
df.shape

<i  class="fa fa-pencil"></i> On peut tout d'abord remarquer que le jeu de données ne semble, a priori, pas idéal pour réaliser des méthodes de machine learning très complexes avec beaucoup de paramètres à estimer. Il arrive très souvent que des méthodes plus classiques soient aussi, voire plus, efficaces que les méthodes d'apprentissage statistique. Cependant ce jeu de données peut tout à fait être utilisé à des fins pédagogiques pour comprendre les principes généraux.

In [ ]:
df.head(10)

Tout d'abord, commençons par définir quelques constantes qui nous seront utiles pour la suite, à savoir : 
- La variable d'intérêt que nous cherchons à prédire `TARGET_VARIABLE`
- La variable correspondant au numéro d'individu `NOIND`
- Un nombre arbitraire pour afin de simplifier la réplicabilité de nos expérimentations `SEED`

In [ ]:
TARGET_VARIABLE="imc"
INDEX="NOIND"
SEED=2023

**Question 1:** Comme souvent en science de la données, la partie la plus fastidieuse consiste à analyser les données à notre disposition. En vous aidant du dictionnaire accessible [ici](https://static.data.gouv.fr/resources/donnees-de-consommations-et-habitudes-alimentaires-de-letude-inca-3/20210128-192017/notice-utilisateurs-donnees-inca3-data.gouvjanv21.pdf) déterminer l'ensemble des variables numériques. Les autres variables seront considérées comme des variables catégorielles.

In [ ]:
NUMERICAL = [
# REMPLIR ICI
]

CATEGORICAL = [x for x in df.columns if x not in NUMERICAL + [INDEX, TARGET_VARIABLE]]

FEATURES = NUMERICAL + CATEGORICAL

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
NUMERICAL = [
    "IA_score",
    "bmr_kcal",
    'nbenf',
    "enceinte_nbmois",
    'nb_cigarettes_jour',
    'nb_cigarettes_sem',
    'nb_cigares_jour',
    'nb_cigares_sem',
    'nb_pipes_jour',
    'nb_pipes_sem',
    'fume_age_debut',
    'fume_age_arret',
    'allaite_nbsem',
    "regime_nb_2dernann",
    "regime_nb_anter2dernann"
]

CATEGORICAL = [x for x in df.columns if x not in NUMERICAL + [INDEX, TARGET_VARIABLE]]

FEATURES = NUMERICAL + CATEGORICAL
```

</details>

Une pratique courante dans les projets de machine learning c'est de commencer par spécifier une fraction de notre jeu données comme un **échantillon de test**. Cet échantillon va être utilisé à la toute fin du projet de sorte à évaluer la performance de nos modèles sur des données qu'il n'aura jamais vu auparavant. L'échantillon restant, celui **d'entrainement**, est lui utilisé pour entrainer les algorithmes et comparer leurs performances. L'idée derrière cette division est de réduire le risque de sur-apprentissage de notre modèle et d'estimer une erreur de généralisation.

**Question 2:** Créer les variables `y` et `X` correspondant respectivement à la variable d'intérêt et aux différentes features de notre jeu de données. Diviser ensuite ce jeu de données en un échantillon de train et de test en utilisant la fonction `train_test_split` de [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Ne pas oublier de spécifier le `random_state`.

In [ ]:
from sklearn.model_selection import train_test_split
# Votre code ici

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
from sklearn.model_selection import train_test_split

X = df[FEATURES]
y = df[TARGET_VARIABLE]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
```

</details>

## 2. Un modèle de régression linéaire simple

**Question 3:** Avant d'étudier différentes méthodes d'apprentissage statistique commençons par réaliser une régression linéaire classique. Pour cela, sélectionnez un petit nombre de variables $(< 10)$ qui vous semble pertinent pour prédire l'indice de masse corporelle d'une personne. Prenez à la fois des variables numériques et catégorielles.

In [ ]:
CATEGORICAL_REGRESSION = [
# A REMPLIR
]

NUMERICAL_REGRESSION = [
# A REMPLIR
]

FEATURES_REGRESSION = NUMERICAL_REGRESSION + CATEGORICAL_REGRESSION

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
CATEGORICAL_REGRESSION = [
    "sex_PS",
    "tage_PS",
   # "tage_PS_mois",
   # "diplome_interv",
   # "soins",
   # "situ_fin_3cl",
    "revenu",
    "situ_alim_statut",
   # "IA_statut",
   # "statnut",
    "poids_perception" ,
   # "menopause",
    "enceinte",
   # "enceinte_12dermois",
   # 'etude_4cl_interv',
   # 'situ_prof_5cl_interv',
   # 'atrav_interv',
   # 'trav_nuit_interv',
   # 'trav_nuit_2cl_interv',
   # 'PCS_8cl_interv',
   # 'PCS_4cl_interv',
   # 'tps_travail_interv',
   # 'vacances_interv',
   # 'RUC_4cl',
]

NUMERICAL_REGRESSION = [
    "IA_score",
   # "bmr_kcal",
   # 'nbenf',
    "enceinte_nbmois",
    "nb_prise_10kg"
]

FEATURES_REGRESSION = NUMERICAL_REGRESSION + CATEGORICAL_REGRESSION
```

</details>

In [ ]:
X_train_regression = X_train[FEATURES_REGRESSION]
X_test_regression = X_test[FEATURES_REGRESSION]

In [ ]:
X_train_regression.info()

Ces informations nous indiquent qu'il y a plusieurs variables qui contiennent des valeurs manquantes. Afin de ne pas supprimer les lignes qui contiennent des valeurs manquantes nous allons tenter de les imputer. Plusieurs méthodes d'imputations peuvent être réalisées :

- **Pour les variables numériques:** Il est courant de remplacer les variables manquantes par la moyenne ou la médiane de l'échantillon.
- **Pour les variables catégorielles:** On peut remplacer les variables manquantes par le moden c'est à dire la valeur la plus fréquente dans l'échantillon ou en créant une nouvelle categorie reflétant une valeur manquante.

D'autres méthodes qui requiert plus de modélisation sont également possible comme réaliser une régression afin de prédire les valeurs manquantes grâce aux autres features ou  utiliser un algorithme de K plus proche voisin. Toutes ces méthodes ont à la fois leurs avantages et leurs inconvénients, il est important de déterminer celle qui est la plus approprié pour le problème que vous souhaitez résoudre.

**Question 4:** Pour faire simple, nous allons remplacer les valeurs manquantes des variables numériques par la médiane et pour celles des variables catégorielles nous allons créer une nouvelle catégorie qui sera égale à $-1$. Ce dernier choix vous semble t-il approprié ?

**Question 5:** En vous aidant de la (documentation)[https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute] de scikit learn , créez ces deux *Imputer*.

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

minus_one_imputer = # YOUR CODE
median_imputer = # YOUR CODE

**Question 6:** Analysez les modalités de la variables `enceinte_nbmois`, une imputation par la moyenne vous semble-t-elle justifiée ? Si non, proposez une autre imputation. Existe t-il d'autres variables dans ce cas ?

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
pd.unique(X_train_regression["enceinte_nbmois"])
```

```python
# Les valeurs manquantes correspondent plutôt à la modalité "pas enceinte", il est donc plus judicieux de remplacer les valeurs manquantes par 0.
zero_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
```

</details>

Une étape très importante lorsqu'on utilise des méthodes de machine learning est la standardisation des données afin de mettre toutes les variables à la même échelle. Lorsque les variables ont des échelles différentes, certaines peuvent dominer les autres dans le processus d'apprentissage, ce qui peut fausser les résultats. Plusieurs méthodes de standardisation peuvent être utilisées, les deux plus courantes sont: 
- la normalisation standard : $z = \frac{x - \bar{x}}{\sigma}$
- la normalisation 0-1 : $z = \frac{x - min}{max - min}$

Nous pouvons donc créer une pipeline dans laquelle nos *features* passeront afin de subir diverses transformations. En l'occurence, nous souhaitons que nos les valeurs manquantes *features* soient imputées et que ces dernières soit standardisées. Pour cela nous pouvons utiliser la fonction `make_pipeline`. Pour le *scaler* nous allons utiliser la normalisation standard qui peut être réalisée grâce à la méthode `StandardScaler()`.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

numerical_zero = make_pipeline(zero_imputer, StandardScaler())
numerical_median = make_pipeline(median_imputer, StandardScaler())
categorical_encoder = make_pipeline(minus_one_imputer, StandardScaler())

Une fois nos *pipelines* définies ils faut déterminer quelles *features* passent par quelles *pipelines*. Dans notre cas, on souhaite que les variables catégorielles traversent la pipeline qui impute les valeurs manquantes par $-1$ et les variables numériques par la pipeline qui impute soit par la médiane, soit par 0. Pour cela on doit utiliser la fonction `ColumnTransformer`. 

**Question 7:** En vous référant à la [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) de la fonction, créez votre pipeline de preprocessing.

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor_regression = ColumnTransformer(
    #YOUR CODE
)

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
from sklearn.compose import ColumnTransformer

preprocessor_regression = ColumnTransformer(
    transformers=[
        ("numerical_zero", numerical_zero, ["enceinte_nbmois", "nb_prise_10kg"]),
        ("numerical_median", numerical_median, [x for x in NUMERICAL_REGRESSION if x not in  ["enceinte_nbmois", "nb_prise_10kg"]]),
        ("categorical", categorical_encoder, CATEGORICAL_REGRESSION)
    ]
)
```

</details>

Maintenant que nos étapes de *preprocessing* sont définies, on peut les réaliser et observer les changements qui ont été opéré sur notre jeu de données afin de vérifier que les modifications ont bien été faites comme attendues. Pour cela, commencons par observer notre échantillon d'entrainement initial.

In [ ]:
X_train_regression

Jusqu'à présent nous avons seulement définies les étapes de notre *preprocessing* mais celles ci n'ont pas été réalisées, pour cela nous devons `fit` notre preprocessing à notre jeu de données.

In [ ]:
preprocessor_regression.fit(X_train_regression)

In [ ]:
preprocessor_regression.get_feature_names_out()

Scikit learn juxtapose automatiquement le nom de la transformation effectuée à la variable. Pour simplifier la comparaison nous allons supprimer ce qui a été rajouté en prefixe de sorte à retrouver les même nom de variable qu'initialement. 

In [ ]:
import re

original_feature_names = [re.sub(r'^.*__', '', item) for item in preprocessor_regression.get_feature_names_out()]
original_feature_names

In [ ]:
data_preprocessed = pd.DataFrame(preprocessor_regression.fit_transform(X_train_regression), columns=original_feature_names)
data_preprocessed

Une fois qu'on a vérifié que le preprocessing nous convient on peut analyser la corrélation des différentes variables explicatives pour se prévenir du problème de la collinéarité. Pour cela, rien de mieux qu'une visualisation graphique pour obtenir une première idée !

**Question 8:** Calculer la matrice de correlation des variables explicatives.

In [ ]:
corr = # YOUR CODE

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
corr = data_preprocessed.corr()
```

</details>

In [ ]:
import seaborn as sns

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
    )

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

Nous pouvons maintenant passer à la modélisation ! Sachant qu'on a déjà une *pipeline* qui contient les instructions pour le preprocessing il est très simple de rajouter une étape supplémentaire à cette pipeline afin de réaliser la modélisation. Pour estimer une régression linéaire nous allons utiliser une nouvelle fois une méthode de scikit-learn: `LinearRegression`. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

pipe_lr = Pipeline([
    ('preprocessor', preprocessor_regression), # 1ère étape réaliser le preprocessing
    ('regression', LinearRegression()) # 2ème étape estime notre régression linéaire
])

Comme précédemment, notre pipeline n'a pas été exécutée, nous l'avons seulement définie. Il est donc nécessaire de l'executer sur nos données grâce à la méthode `fit`.

In [ ]:
lr = pipe_lr.fit(X_train_regression, y_train)

**Question 9:** Prédisez les indices de masse corporelles des individus de votre échantillon de test à l'aide de votre modèle. Evaluez le en calculant l'écart quadratique moyen (RMSE) et le R2.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = # YOUR CODE
rmse = # YOUR CODE
r2 = # YOUR CODE
print(f"Le RMSE sur le jeu de test est : {round(rmse, 4)}")
print(f"Le R2 sur le jeu de test est : {round(r2, 4)}")

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
from sklearn.metrics import mean_squared_error

y_pred = lr.predict(X_test_regression)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Le RMSE sur le jeu de test est : {round(rmse, 4)}")
print(f"Le R2 sur le jeu de test est : {round(r2, 4)}")
```

</details>

Il est difficile d'interpréter la valeur absolue du RMSE car il dépend de l'échelle et de la volatilité des données que nous cherchons à prédire. Regardons quelques statistiques de nl'IMC de nos individus de l'échantillons de test.

In [ ]:
print(f"La variance de l'IMC de jeu de test est : {round(y_test.var(), 4)}")
print(f"La moyenne de l'IMC de jeu de test est : {round(y_test.mean(), 4)}")
print(f"L'écart interquartile de l'IMC de jeu de test est : {round(y_test.quantile(0.75) - y_test.quantile(0.25), 4)}")


On peut donc normaliser notre RMSE par l'une de ces statistiques pour avoir une idée plus précise des erreurs. La variance et la moyenne sont généralement les plus utilisées pour normaliser le RMSE.

In [ ]:
print(f"Le RMSE sur le jeu de test équivaut à {round(rmse / y_test.var() * 100 , 2)}% de la variance de l'échantillon.")


Une autre méthode qu'on ne saurait que vous recommander est une nouvelle fois la représentation graphique ! 

**Question 10:** Représentez graphiquement les valeurs prédites par rapport aux vraies valeurs.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
import matplotlib.pyplot as plt

# Creer le scatter plot
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)

# Ajout des labels
plt.xlabel('Vrai IMC')
plt.ylabel('IMC prédit')

# Ajout de la ligne à 45° comme référence
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

# Affichagedu graphique
plt.show()
```

</details>

**Question 11:** Commentez le graphique.

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, RidgeCV, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

In [ ]:
#CATEGORICAL_REGRESSION_IMPUT = ["tage_PS_mois", "diplome_interv", "poids_perception", "menopause", "enceinte", "enceinte_12dermois", "atrav_interv", "trav_nuit_interv", "trav_nuit_2cl_interv", "RUC_4cl", "tps_travail_interv"]

## 3. Méthodes de Machine Learning

### 3.1 Random Forest Regression


Grâce à scikit learn, on va voir qu'il est très facile d'utiliser des méthodes de machine learning différentes maintenant qu'on a utilisé les fonctions de base lors de la régression linéaire. 
Nous allons maintenant essayer d'utiliser l'intégralité des variables qui nous sont disponibles dans la base de données initiale. Cela implique de se replonger un petit peu dans l'analyse des données. Nous avons vu précédemment que toutes les variables numériques ne pouvaient pas être imputées par la médiane et que dans certains cas une imputation par 0 est préférable. 

**Question 12:** Répertoriez l'ensemble des variables numériques à imputer par 0 dans une liste.

In [ ]:
NUMERICAL_0_IMPUT = [
    "enceinte_nbmois", "nb_prise_10kg", "nb_cigarettes_jour",  
    "nb_cigarettes_sem", "nb_cigares_jour", "nb_cigares_sem", 
    "nb_pipes_jour", "nb_pipes_sem", "allaite_nbsem", 
    "regime_nb_2dernann", "regime_nb_anter2dernann"
    ]

<details>
<summary>
    <font size=3 color="red"><b>Click to see the answer </b></font>
</summary>

```python
NUMERICAL_0_IMPUT = [
    "enceinte_nbmois", "nb_prise_10kg", "nb_cigarettes_jour",  
    "nb_cigarettes_sem", "nb_cigares_jour", "nb_cigares_sem", 
    "nb_pipes_jour", "nb_pipes_sem", "allaite_nbsem", 
    "regime_nb_2dernann", "regime_nb_anter2dernann"
    ]
```

</details>

In [ ]:
NUMERICAL_0_IMPUT = ["enceinte_nbmois", "nb_prise_10kg", "nb_cigarettes_jour",  "nb_cigarettes_sem", "nb_cigares_jour", "nb_cigares_sem", "nb_pipes_jour", "nb_pipes_sem", "allaite_nbsem", "regime_nb_2dernann", "regime_nb_anter2dernann"]

In [ ]:
zero_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
minus_one_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1)

numerical_zero = make_pipeline(zero_imputer, StandardScaler())
numerical_median = make_pipeline(median_imputer, StandardScaler())

categorical_encoder = make_pipeline(minus_one_imputer)

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical_zero", numerical_zero, NUMERICAL_0_IMPUT),
        ("numerical_median", numerical_median, [x for x in NUMERICAL if x not in NUMERICAL_0_IMPUT]),
        ("categorical", categorical_encoder, CATEGORICAL)
    ]
)


In [ ]:
pipe_rfr = Pipeline([
    ('preprocessor', preprocessor), 
    ('regression', RandomForestRegressor(random_state=SEED))
])


In [ ]:
pipe_rfr['regression'].get_params()


In [ ]:
pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())

On fait un grid search

In [ ]:
param_grid = {
    "regression__n_estimators": [50, 100, 200],
    "regression__max_leaf_nodes": [5, 10, 50]
}

In [ ]:
pipe_gscv = GridSearchCV(pipe_rfr, 
                         param_grid=param_grid, 
                         scoring=["explained_variance", "r2", "neg_root_mean_squared_error"],
                         refit="neg_root_mean_squared_error",
                         cv=5, 
                         n_jobs=-1, 
                         verbose=1)

Combien on va faire de simulation ?

In [ ]:
rfr = pipe_gscv.fit(X_train, y_train)


In [ ]:
perf_random_forest = pd.DataFrame(rfr.cv_results_)
perf_random_forest

In [ ]:
fig, ax = plt.subplots(1)

# Line plot
for max_leaf_nodes, group in perf_random_forest.groupby("param_regression__max_leaf_nodes"):
    x = group["param_regression__n_estimators"]
    y = group["mean_test_neg_root_mean_squared_error"]
    ax.scatter(x, y, label=f"{max_leaf_nodes}")

# Set labels and title
ax.set(xlabel='Number of estimators', ylabel='Mean score on test sample')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0,
           title='Depth of trees')
# Display the chart
plt.show()


In [ ]:
y_pred = rfr.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Le RMSE sur le jeu de test est : {round(rmse, 4)}")

In [ ]:
# Create the scatter plot
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)

# Add labels and title
plt.xlabel('Actual IMC')
plt.ylabel('Predicted IMC')
plt.title('Scatter Plot of Predicted IMC vs Actual IMC')

# Add a diagonal line for reference
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

# Display the plot
plt.show()


# 4. SVM

In [ ]:
from sklearn.svm import SVR


In [ ]:
zero_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
minus_one_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1)

numerical_zero = make_pipeline(zero_imputer, StandardScaler())
numerical_median = make_pipeline(median_imputer, StandardScaler())

categorical_encoder = make_pipeline(minus_one_imputer)

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical_zero", numerical_zero, NUMERICAL_0_IMPUT),
        ("numerical_median", numerical_median, [x for x in NUMERICAL if x not in NUMERICAL_0_IMPUT]),
        ("categorical", categorical_encoder, CATEGORICAL)
    ]
)


In [ ]:
pipeline_svr = Pipeline([
    ('preprocessor', preprocessor), 
    ('regression', SVR())
])

In [ ]:
param_grid = {
    "regression__C": np.logspace(-8, 8, 9, base=2), 
    "regression__kernel": ["rbf"],
    "regression__gamma": [0.01],
}

In [ ]:
pipe_gscv = GridSearchCV(pipeline_svr, 
                         param_grid=param_grid, 
                         scoring=["explained_variance", "r2", "neg_root_mean_squared_error"],
                         refit="neg_root_mean_squared_error",
                         cv=5, 
                         n_jobs=-1, 
                         verbose=1)

In [ ]:
# Fit the pipeline to the training data
pipe_gscv.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipe_gscv.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# Create the scatter plot
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)

# Add labels and title
plt.xlabel('Actual IMC')
plt.ylabel('Predicted IMC')
plt.title('Scatter Plot of Predicted IMC vs Actual IMC')

# Add a diagonal line for reference
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

# Display the plot
plt.show()


# 5. XGboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor), 
    ('regression', XGBRegressor())
])

In [ ]:
pipeline_xgb['regression'].get_params()


In [ ]:
param_grid = {
    "regression__max_leaves": [5, 10, 25],
    "regression__max_depth": [2, 3, 5],
    "regression__gamma": [0.1, 0.25, 0.5],
    "regression__n_estimators": [75, 150, 250],
}

In [ ]:
pipe_gscv = GridSearchCV(pipeline_xgb, 
                         param_grid=param_grid, 
                         scoring=["explained_variance", "r2", "neg_root_mean_squared_error"],
                         refit="neg_root_mean_squared_error",
                         cv=5, 
                         n_jobs=-1, 
                         verbose=1)

In [ ]:
# Fit the pipeline to the training data
pipe_gscv.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipe_gscv.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# Create the scatter plot
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)

# Add labels and title
plt.xlabel('Actual IMC')
plt.ylabel('Predicted IMC')
plt.title('Scatter Plot of Predicted IMC vs Actual IMC')

# Add a diagonal line for reference
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

# Display the plot
plt.show()

# 6. KNeighbors

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
pipeline_knr = Pipeline([
    ('preprocessor', preprocessor), 
    ('regression', KNeighborsRegressor())
])

In [ ]:
pipeline_knr['regression'].get_params()


In [ ]:
param_grid = {
    "regression__n_neighbors": [3, 4, 5, 6],
    "regression__p": [1, 2, 3],
}

In [ ]:
pipe_gscv = GridSearchCV(pipeline_knr, 
                         param_grid=param_grid, 
                         scoring=["explained_variance", "r2", "neg_root_mean_squared_error"],
                         refit="neg_root_mean_squared_error",
                         cv=5, 
                         n_jobs=-1, 
                         verbose=1)

In [ ]:
# Fit the pipeline to the training data
pipe_gscv.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipe_gscv.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# Create the scatter plot
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)

# Add labels and title
plt.xlabel('Actual IMC')
plt.ylabel('Predicted IMC')
plt.title('Scatter Plot of Predicted IMC vs Actual IMC')

# Add a diagonal line for reference
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

# Display the plot
plt.show()

# ANNEXES

In [ ]:
list_var = [
"NOIND",
"imc",
"sex_PS",
"tage_PS",
"tage_PS_mois",
"diplome_interv",
"etude_4cl_interv",
"situ_prof_5cl_interv",
"atrav_interv",
"trav_nuit_interv",
"trav_nuit_2cl_interv",
"PCS_8cl_interv",
"PCS_4cl_interv",
"tps_travail_interv",
"vacances_interv",
"soins",
"situ_fin_3cl",
"revenu",
"RUC_4cl",
"nbpers",
"nbadu",
"nbenf",
"situ_alim_statut",
"IA_statut",
"IA_score",
"statnut",
"maladie_allergie_alim",
"intoall_confirm_med",
"regime_vegetarien",
"regime_allergie",
"regime_maigrir_med",
"regime_maigrir_choix",
"regime_autre_med",
"regime_poidsstable",
"regime_forme",
"regime_autreraison",
"regime_non",
"veget_viande",
"veget_prodmer",
"veget_prodlait",
"veget_oeuf",
"veget_miel",
"veget_autre_alim",
"allergie_laitvache",
"allergie_prepainfsoja",
"allergie_prepainfamande",
"allergie_gluten",
"allergie_farineble",
"allergie_lupin",
"allergie_arachide",
"allergie_fruitcoque",
"allergie_oeuf",
"allergie_poisson",
"allergie_crustace",
"allergie_mollusque",
"allergie_soja",
"allergie_sesame",
"allergie_moutarde",
"allergie_sulfite",
"allergie_celeri",
"allergie_autres_fruitleg",
"allergie_autresalim",
"allergie_nondetermine",
"allergie_fruits",
"allergie_legumes",
"regime_passe",
"regime_nb_2dernann",
"regime_nb_anter2dernann",
"regime_type" ,
"regime_duree_sem",
"regime_duree_mois",
"regime_duree_nsp" ,
"poids_modif",
"poids_modifalim",
"poids_plusAP",
"poids_medicaments",
"poids_substituts",
"poids_chirurgie",
"poids_modifalim_laityaourt",
"poids_modifalim_fromage",
"poids_modifalim_mg",
"poids_modifalim_fruit",
"poids_modifalim_legume",
"poids_modifalim_pdtfeculent",
"poids_modifalim_pizza",
"poids_modifalim_pain",
"poids_modifalim_vrouge",
"poids_modifalim_volaille",
"poids_modifalim_oeuf",
"poids_modifalim_gateau",
"poids_modifalim_edulcorant",
"poids_modifalim_pdtsalleges",
"poids_modifalim_BS",
"poids_modifalim_eau",
"poids_modifalim_autre",
"poids_perception" ,
"nb_prise_10kg",
"menopause",
"enceinte",
"enceinte_nbmois",
"allaite",
"allaite_nbsem",
"enceinte_12dermois",
"fume",
"nb_cigarettes_jour",
"nb_cigarettes_sem",
"nb_cigarettes_nsp",
"nb_cigares_jour",
"nb_cigares_sem",
"nb_cigares_nsp",
"nb_pipes_jour",
"nb_pipes_sem",
"nb_pipes_nsp",
"fume_age_debut",
"fume_age_debut_nsp",
"fume_age_arret",
"fume_age_arret_nsp",
"bmr_kcal"]


In [ ]:
new_data = desc_indiv[list_var]

In [ ]:
new_data = new_data.dropna(subset=['imc'])

In [ ]:
new_data.to_parquet("description_individu_inca.parquet")